# Installing all the needed packages (Run this only if you're using the Google Collab)

In [1]:
# apt-get handle the installation and removal of software on Debian, Ubuntu, and related Linux distributions
!apt-get update

# installing needed packages
!apt install chromium-chromedriver
!pip install selenium
!pip install beautifulsoup4

# Copy the chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
# This make sure that the selenium chromedriver is in the path
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://sec

# Settings all imports

In [2]:
import tqdm
import requests
import urllib.request

import numpy as np
import pandas as pd

from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from bs4.element import Comment
from google.colab import drive

# Moutning Google Drive device

In [3]:
drive.mount("/content/drive")

Mounted at /content/drive


# Create a web driver based on operating system. Please change the value of the `OPERATING_SYSTEM` constant to run this notebook on your system

In [4]:
OPERATING_SYSTEM = "linux"

if OPERATING_SYSTEM == "windows":
    executable_path = "drivers/windows/chromedriver.exe"
elif OPERATING_SYSTEM == "mac":
    executable_path = "drivers/mac/chromedriver"
elif OPERATING_SYSTEM == "linux":
    executable_path = "chromedriver"

base_url = 'https://www.jobs.bg/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(executable_path=executable_path, options=chrome_options)

url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
r = driver.get(url)
html = driver.execute_script("return document.documentElement.outerHTML")
soup = BeautifulSoup(html, 'html.parser')

In [6]:
def tag_visible(element):
    """
    """
    not_visible_elements = ['style', 'script', 'head', 'title', 'meta']
    if element.parent.name in not_visible_elements:
        return False

    if isinstance(element, Comment):
        return False

    return True

def text_from_soup(soup):
    """
    """
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    text_string = u" ".join(visible_text.strip() for visible_text in visible_texts)
    text_string = text_string.strip()
    return text_string

def text_from_html(body):
    """
    """
    soup = BeautifulSoup(body, 'html.parser')
    text_string = text_from_soup(soup)
    return text_string

def extract_job_text(position_soup):
    """
    """
    job_text = np.nan

    iframe_elem = position_soup.find('iframe', attrs={'id': 'customJobIframe'})
    raw_html_elem = position_soup.find('td', attrs={'class': 'jobTitle'})

    if iframe_elem:
        iframe_url = iframe_elem.attrs['src']
        job_http_response = urllib.request.urlopen(iframe_url)
        job_text = text_from_html(job_http_response).lstrip('html ')
    elif raw_html_elem:
        raw_html_elem = raw_html_elem.parent.parent
        raw_soup = BeautifulSoup(raw_html_elem.text, 'html.parser')
        job_text = text_from_soup(raw_soup)
  
    return job_text


In [15]:
position_url = 'https://www.jobs.bg/job/5837867'
r = driver.get(position_url)
html = driver.execute_script("return document.documentElement.outerHTML")
p_soup = BeautifulSoup(html, 'html.parser')



extract_job_text(p_soup)

"html     Requisition ID: 278246 Work Area: Software-Development Operations Expected Travel: 0 - 10% Career Status: Professional Employment Type: Regular Full Time COMPANY DESCRIPTION SAP started in 1972 as a team of five colleagues with a desire to do something new. Together, they changed enterprise software and reinvented how business was done. Today, as a market leader in enterprise application software, we remain true to our roots. That’s why we engineer solutions to fuel innovation, foster equality and spread opportunity for our employees and customers across borders and cultures. SAP values the entrepreneurial spirit, fostering creativity and building lasting relationships with our employees. We know that a diverse and inclusive workforce keeps us competitive and provides opportunities for all. We believe that together we can transform industries, grow economics, lift up societies and sustain our environment. Because it’s the best-run businesses that make the world run better and

In [36]:
iframe_url = 'https://www.jobs.bg/job_view_sandboxed.php?job_sid=5894567'
job_http_response = urllib.request.urlopen(iframe_url)
job_http_response
job_text = text_from_html(job_http_response)
job_text


"html     Abalta Technologies is looking for experienced C++ engineers to join our products team. Ideal candidates are creative self-starters with great drive and motivation who want to work on exciting products, eager to push the limits of the car infotainment while competing with the best companies in the industry. If you are up for the challenge send us your resume and let's talk. We offer:  Career path and opportunity to learn and grow  Competitive salary with annual updates  Work on projects with top international companies on a multicultural team  Modern office close to a gym and food courts  Flexible working hours with 25 days paid time off  Food vouchers  Company sponsored transit pass or parking  Team buildings and corporate parties  Top tier hot beverages ( we do care about the quality )  The skill set we are looking for:  Profficiency in C++  Have at least 2 years of experience in developing software solutions using C++ and Object Oriented Design  Have knowledge of managing 

In [28]:
def tag_visible(element):
    """
    """
    not_visible_elements = ['style', 'script', 'head', 'title', 'meta']
    if element.parent.name in not_visible_elements:
        return False

    if isinstance(element, Comment):
        return False

    return True

def text_from_soup(soup):
    """
    """
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    text_string = u" ".join(visible_text.strip() for visible_text in visible_texts)
    text_string = text_string.strip()
    return text_string

def text_from_html(body):
    """
    """
    soup = BeautifulSoup(body, 'html.parser')
    text_string = text_from_soup(soup)
    return text_string

def extract_technologies(text_info_elem):
    """
    """
    technologies = []

    for a in text_info_elem.find_all('img'):
        current_technology = a.attrs['alt']
        technologies.append(current_technology)

    technologies = ", ".join(technologies)

    return technologies

def extract_rating_color(rating_elem):
    """
    """
    rating_color = np.nan
  
    if rating_elem:
        rataing_elem_styles = rating_elem["style"].split(";")
        rating_elem_color_styles = [x for x in rataing_elem_styles if "color:" in x]
        rating_color = rating_elem_color_styles[0].split(":")[1]
    
    return rating_color

def extract_rating_value(rating_elem, full_start_string="\uf4b3", half_start_string="\uf4b1"):
    """
    """
    rating = np.nan

    if rating_elem:
        full_stars_count = rating_elem.text.strip().count(full_start_string)
        half_stars_count = rating_elem.text.strip().count(half_start_string)
        rating = full_stars_count + (half_stars_count / 2)

    return rating

def extract_job_date(text_info_elem):
    """
    """
    class_date_elem = "card__subtitle mdc-typography mdc-typography--overline line-height-1-5"
    date_elem = text_info_elem.find('span', attrs={'class': class_date_elem})
  
    if date_elem:
        job_date = date_elem.text.strip()

        if job_date == 'днес':
            job_date = date.today().strftime("%d.%m.%y")
        elif job_date == 'вчера':
            job_date = (date.today() - timedelta(days=1)).strftime("%d.%m.%y")
    
        return job_date

    return np.nan

def extract_job_text(position_soup):
    """
    """
    job_text = np.nan

    iframe_elem = position_soup.find('iframe', attrs={'id': 'customJobIframe'})
    raw_html_elem = position_soup.find('td', attrs={'class': 'jobTitle'})

    if iframe_elem:
        iframe_url = iframe_elem.attrs['src']
        job_http_response = urllib.request.urlopen(iframe_url)
        job_text = text_from_html(job_http_response).lstrip('html ')
    elif raw_html_elem:
        raw_html_elem = raw_html_elem.parent.parent
        raw_soup = BeautifulSoup(raw_html_elem.text, 'html.parser')
        job_text = text_from_soup(raw_soup)
  
    return job_text

def get_pages(soup):
    """Get the total number of job offerings and the total number of pages.
    output: total number of pages
    """
    td_searched_style = "height:25px;width:220px;font-weight:500;padding-bottom:5px;"
    total_job_number = soup.find('td', attrs={'style': td_searched_style}).text.split()[-1]
    total_job_number = int(total_job_number)
    total_pages = int(total_job_number / 15) + 1 # round up

    return total_pages

def get_position_id(text_info_elem):
    """
    """
    class_id_elem = 'card__title mdc-typography mdc-typography--headline6 text-overflow'
    id_elem = text_info_elem.find('a',attrs={'class':class_id_elem})['href']
    position_id = id_elem.lstrip('job/')
    
    return position_id

def get_position_url(base_url, text_info_elem):
    """
    """
    class_url_elem = 'card__title mdc-typography mdc-typography--headline6 text-overflow'
    url_elem = text_info_elem.find('a',attrs={'class':class_url_elem})
    job_url = url_elem['href'] 
    position_url = base_url + job_url
    
    return position_url

def get_position_name(text_info_elem):
    """
    """
    class_name_elem = 'card__title mdc-typography mdc-typography--headline6 text-overflow'
    name_elem = text_info_elem.find('a', attrs={'class': class_name_elem})
    position_name = name_elem.text.strip()
    
    return position_name

def get_benefits(text_info_elem):
    """
    """
    class_benefit_elem = 'card__subtitle mdc-typography mdc-typography--body2 top-margin'
    benfit_elem = text_info_elem.find('span', attrs={'class': class_benefit_elem})
    position_benefit = benfit_elem.text.strip()
    
    return position_benefit

def get_position_views(position_soup):
    """
    """
    span_string_elem = 'Разглеждания:'
    string_elem = position_soup.find('span', string = span_string_elem)
    position_views = string_elem.parent.text.split()[-1]
    
    return position_views    
    
def extract_job_info_as_array(base_url, text_info_elem, job_company_elem):
    """
    """
    position_id = get_position_id(text_info_elem)
    position_url = get_position_url(base_url,text_info_elem)
    company_name = job_company_elem.text.strip()

    _ = driver.get(position_url)
    job_html = driver.execute_script("return document.documentElement.outerHTML")
    position_soup = BeautifulSoup(job_html, 'html.parser')

    position_views = get_position_views(position_soup)
    job_text = extract_job_text(position_soup)
    job_date = extract_job_date(text_info_elem)

    rating_elem = text_info_elem.find('span', attrs={'class':'iconed'})
    rating_color = extract_rating_color(rating_elem)
    rating = extract_rating_value(rating_elem)

    position = get_position_name(text_info_elem)
    benefits = get_benefits(text_info_elem)
    technologies_string = extract_technologies(text_info_elem)

    current_job_info = [position_id, position_url, company_name, position_views, job_text, 
                        job_date, rating, rating_color, position, benefits, technologies_string]
    
    return current_job_info

In [29]:
def extract_jobs(start_page, end_page, category_id):
    """"""
    jobs_list = []

    for page in tqdm.tqdm(range(start_page, end_page)):

        try:
            # This is the base url used when no filters by categories are applied
            url = "https://www.jobs.bg/front_job_search.php?frompage={}&add_sh=1&from_hp=1&term=#paging".format(page*15)
          
            if category_id:
                url = "https://www.jobs.bg/front_job_search.php?frompage={}&add_sh=1&categories%5B0%5D={}&term=#paging".format(page*15, category_id)
          
            r = driver.get(url)
            html = driver.execute_script("return document.documentElement.outerHTML")
            soup = BeautifulSoup(html, 'html.parser')
            job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})
        except Exception as e:
            print(f"Page number: {page}, url: {url}")
            print(f"Unexpected error: {e.message}")

        for i in range(0, 45, 3):
            try:
                text_info_elem = job_infos[i]
                job_logo_elem = job_infos[i+1]
                job_company_elem = job_infos[i+2]

                current_job_info = extract_job_info_as_array(base_url, text_info_elem, job_company_elem)
                jobs_list.append(current_job_info)
            except Exception as e:
                print(f"Curent page: {page}, problem with the {i}, {i+1}, {i+2} tds")
                print(f"Unexpected error: {e}")

    df_columns = ["position_id", "position_url", "company_name", "position_views", "job_description",
                  "date_posted", "position_rating", "rating_color", "position", "benefits", "technologies"]

    jobs_df = pd.DataFrame(jobs_list, columns=df_columns)

    return jobs_df

# Extracting the category names and ids so we can perform filters based on category

In [30]:
def extract_categories_dataframe():
    """"""
    url = "https://www.jobs.bg/index.php"
    r = driver.get(url)
    categories_button = driver.find_element_by_id("categoriesChip")
    categories_button.click()

    html = driver.execute_script("return document.documentElement.outerHTML")
    soup = BeautifulSoup(html, 'html.parser')

    categories_elem = soup.find('div',attrs={'id':'categoriesSelectSheet'})
    categories_elements = categories_elem.find_all("div", attrs={"class": "mdc-chip mdc-ripple-upgraded"})

    categories_info = []
    categories_info.append((56, "IT JOBS"))

    for category_element in categories_elements:
        category_id = category_element['id'].split("_")[-1]
        category_name = category_element.text.replace("\n", "").strip()
        category_info = (category_id, category_name)
        categories_info.append(category_info)

    categories_info = pd.DataFrame(categories_info, columns=["category_id", "category_name"])

    return categories_info

In [31]:
categories_info = extract_categories_dataframe()
categories_info

,category_id,category_name
0,56,IT JOBS
1,45,Търговия и Продажби
2,57,Производство
3,36,"Ресторанти, Заведения, Хотели, Туризъм"
4,38,"Административни, Офис и Бизнес дейности"
5,6,"Шофьори, Куриери"
6,29,Инженери и Техници
7,52,Физически/Ръчен труд
8,3,"Архитектура, Строителство"
9,58,"Логистика, Спедиция"


In [32]:
df = extract_jobs(start_page=0, end_page=10, category_id=56)
df

100%|██████████| 10/10 [03:36<00:00, 21.70s/it]


,position_id,position_url,company_name,position_views,job_description,date_posted,position_rating,rating_color,position,benefits,technologies
0,5894567,https://www.jobs.bg/job/5894567,Абалта Технолоджис ЕООД,399,Abalta Technologies is looking for experienced...,17.09.21,5.0,#66c1ff,C++ Software Developer,София; Възможност за работа от вкъщи; Дистанци...,"C/C++, Jenkins"
1,5876860,https://www.jobs.bg/job/5876860,Абалта Технолоджис ЕООД,270,Abalta Technologies is looking for experienced...,17.09.21,5.0,#66c1ff,Android Developer,София; Възможност за работа от вкъщи; Дистанци...,Android
2,5894573,https://www.jobs.bg/job/5894573,Абалта Технолоджис ЕООД,218,Abalta Technologies is on the leading edge of ...,17.09.21,5.0,#66c1ff,iOS Developer,София; Възможност за работа от вкъщи; Дистанци...,"iOS, Swift, C/C++, Objective-C, Docker, AWS"
3,5894579,https://www.jobs.bg/job/5894579,Абалта Технолоджис ЕООД,660,You have experience working in the field of so...,17.09.21,5.0,#66c1ff,QA Engineer,София; Възможност за работа от вкъщи; Дистанци...,
4,5894582,https://www.jobs.bg/job/5894582,Абалта Технолоджис ЕООД,249,Abalta Technologies is looking for an experien...,17.09.21,5.0,#66c1ff,Technical recruiter,София; Възможност за работа от вкъщи; Дистанци...,"Android, C/C++, Java, Objective-C, Swift, Java..."
...,...,...,...,...,...,...,...,...,...,...,...
145,5908870,https://www.jobs.bg/job/5908870,Paysafe Bulgaria EOOD,129,Cyber Developer Paysafe Limited (“Paysafe”) (...,16.09.21,5.0,#66c1ff,Cyber Developer,София; Възможност за работа от вкъщи; Дистанци...,
146,5908942,https://www.jobs.bg/job/5908942,TTEC Eastern Europe EAD,98,We’re hiring and are looking to connect with y...,16.09.21,5.0,#66c1ff,Technical Support Specialist fluent in Portugu...,София; Възможност за работа от вкъщи; Дистанци...,
147,5908823,https://www.jobs.bg/job/5908823,Paysafe Bulgaria EOOD,127,"Technical Support Specialist, Skrill & NETELLE...",16.09.21,5.0,#66c1ff,"Technical Support Specialist, Skrill & NETELLER",София; Възможност за работа от вкъщи; Дистанци...,"JavaScript, PHP, Java, HTML/CSS"
148,5908809,https://www.jobs.bg/job/5908809,УЛТРАФЛЕКС КОРПОРЕЙШЪН ООД,64,EMBEDDED C/C++ SOFTWARE ENGINEER (FIRMWARE DEV...,16.09.21,5.0,#66c1ff,EMBEDDED C/C++ SOFTWARE ENGINEER (FIRMWARE DE...,София,C/C++


In [ ]:
# df.to_csv("/content/drive/My Drive/jobsbg_scrapped/it_jobs_20210509.csv", index=False)